<h1>웹 크롤링</h1>

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import time

import pandas as pd

In [30]:
def get_data(url):
    #웹 드라이버
    driver = webdriver.Chrome(executable_path='chromedriver')
    driver.implicitly_wait(5)
    driver.get(url)
    while True:
        try:
            더보기 = driver.find_element_by_css_selector('a.u_cbox_btn_more')
            더보기.click()
            time.sleep(1)
        except:
            break
    print("끝")
    
    contents = driver.find_elements_by_css_selector('span.u_cbox_contents')
    # for content in contents:
    #     print(content.text)
    contents = [content.text for content in contents]
    #작성자 추출
    nicks = driver.find_elements_by_css_selector('span.u_cbox_nick')
    # for nick in nicks:
    #     print(nick.text)
    nicks = [nick.text for nick in nicks]
    #날짜 추출
    dates = driver.find_elements_by_css_selector('span.u_cbox_date')
    # for date in dates:
    #     print(date.text)
    dates = [date.text for date in dates]

    #취합
    data = [data for data in list(zip(nicks,dates, contents))]
    driver.quit()
    return data

In [31]:
if __name__ == '__main__':
    url = "https://news.naver.com/main/read.nhn?m_view=1&includeAllCount=true&mode=LSD&mid=shm&sid1=100&oid=422&aid=0000430957"
    data = get_data(url)
    # print(data)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
끝


In [32]:
data[:5]

[('kwgs****',
  '2020.06.02. 14:18',
  '국회의원 말도 안됩니다.퇴출시켜야 합니다.모든 국민의 생각입니다.민주당은 누구를 대변하는겁니까?'),
 ('clse****', '2020.05.30. 21:09', '난 민주당 찍었지만 윤미향은 퇴출이 맞다고 본다'),
 ('dlar****', '2020.05.30. 21:01', '윤미향 쉴드치는 민주당의원들도 같이 퇴출운동 해야한다.'),
 ('krtt****',
  '2020.05.30. 23:21',
  '기부금과 보조금을 횡령한 도둑년이 국회의원이라다 이게 더불어민주당의 수준이다 국민들은 저런 도둑년을 국회의원으로 뽑은적없다 반드시 퇴출시켜야한다'),
 ('cott****',
  '2020.05.30. 20:26',
  '30년동안 이용해먹다가 ㅡ 빨만큼 빨았으니까ㅡ이제 이용가치가 떨어지니 비례대포국개의원으로? ㅡ 정의연은 해체해라!~ 해 먹을꺼도없잔아? ㅡ 토착왜구라 몰아세우더니ㅡ 왜구의 과자맛은 못있었나 보구나?... 국민들이 나서야합니다. 나서서 윤미향이 사퇴시켜야됩니다~ 이런뇬이 뭔 일을 할까요? ㅡ 뻔하지않습니까? ㅡ 빨대 빠는것만 하겠지요!~')]

In [33]:
col = ["작성자","날짜","댓글내용"]
data_frame=pd.DataFrame(data,columns=col)
data_frame.to_csv("new_crawling.csv",mode='w',index = False)

<h1>모델학습</h1>

<h1>데이터 평가</h1>

In [34]:
#데이터 불러오기
dataset = pd.read_csv("new_crawling.csv", index_col=0)
test = [test for test in dataset["댓글내용"]]
print("댓글 갯수 : ",len(dataset["댓글내용"]))
# test

댓글 갯수 :  433


In [47]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
MODEL_NAME = "jihoonkimharu/bert-base-klue-ynat-finetuned"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/563 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/995 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

In [48]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model = model, tokenizer = tokenizer)

In [49]:
# classifier를 통한 긍부정 평가 진행
answer_data = [classifier(answer_data) for answer_data in test]
# answer_data

In [50]:
answer = [data for data in list(zip(test, answer_data))]

In [51]:
#결과 csv파일 만들기.
col = ["댓글 내용","긍부정 평가 결과"]
data_frame=pd.DataFrame(answer,columns=col)
data_frame.to_csv("answer.csv",mode='w',index = False)

In [40]:
# import collections
# # collections.Counter(nicks) 키 벨류 쌍으로 묶어서 나타내줌.
# print(collections.Counter(nicks).values())
# print(len(collections.Counter(nicks).values()))

In [41]:
#matplotlib으로도 시각화해보기
# import matplotlib as plt
# plt.hist(nicks)

# plt.show()